In [1]:
'''
Based on https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html

- useful:
1. https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780
2. https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html
3. https://arxiv.org/pdf/1609.02907.pdf
'''
from networkx import karate_club_graph
from sklearn import preprocessing
from torch_geometric.nn import GCNConv
import torch
import itertools
import torch.nn as nn
from torch_geometric.utils import from_networkx
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.utils import tensorboard
from datetime import datetime

## 0. What are Graph Convolutions? 

### 0.1 The graph Laplacian

The graph Laplacian is defined as:
$$
L \equiv D - A \, , \quad D_{ij} =  \delta_{ij}\sum_{k=0}^{n-1}A_{k,j} = \begin{bmatrix}
    \sum_{k=0}^{n-1}A_{k,0} & & 0 \\
    & \ddots & \\
    0 & & \sum_{k=0}^{n-1}A_{k,n-1}
  \end{bmatrix} \, ,
$$

where $A$ is the *adjacency matrix* and $D$ is *the degree matrix*. The reasons why this is called Laplacian is because this matrix reproduces an analogous behaviour in discrete analysis as the operator $\nabla^2$ in real analysis. Some of the examples where this is shown can be found in wikipedia or in the references.

The normalized Laplacian $L \to D^{-1/2}L D^{-1/2} = I_n - D^{-1/2}A D^{-1/2} $ is more convenient to use because it does not change the scale of the input. From now on we will use $L$ to refer to the normalized Laplacian.

Because $L$ is a symmetric and real valued matrix there exists an orthonormal basis of eigenvectors that diagonalize it, and therefore there is an orthogonal matrix $U$ that diagonalizes the Laplacian matrix:

$$
L = U^T \Lambda\, U\, , \quad \Lambda = \mathrm{diag}(\lambda_0, \dots, \lambda_{n-1}) =   \begin{bmatrix}
    \lambda_{0} & & 0 \\
    & \ddots & \\
    0 & & \lambda_{n-1}
  \end{bmatrix}
  \, ,
$$
with
$$
 U^T U = I_n \iff U_{ik}U_{jk} =  \delta_{ij}
$$


### 0.2 The Graph Fourier Transform

Apparently the values of the eigenvectors of the Laplacian matrix form an oscillatory behaviour emulating the oscillatory character of a wave, from there they conclude that expanding any vector in terms of the Laplacian eigenbasis is the analog of a fourier decomposition of a function. 

I have to say that all this looks all a bit sketchy (probably more reading is necessary) but  thats the main motivation behind the fourier transform I have found so far.

The eigenvectors $\{\mathbf{u}_l\}_{l=0}^{n-1}$ of $L$ are known as the graph Fourier modes and its associated eigenvalues $\{\lambda_l\}_{l=0}^{n-1}$ as the frequencies of the graph.

A signal $x: V \rightarrow \mathbb{R}$ defined on the nodes of the graph may be regarded as a vector $x \in \mathbb{R}^n$, where $x(i)$ is the value of $x$ at the $i^{th}$ node.

Motivated by the above argument the Graph Fourier Transform over a function $f: V \rightarrow \mathbb{R}$
is defined as:

$$
\mathcal{G F}[f]=\hat{f}= U \cdot f
$$

To find the matrix $U$ for small graphs is no problem, but for graphs with around 100 000 nodes is already a challenging (computationally speaking) task. Many graphs of interest in industry contain $\sim 1\, 000\, 000$ nodes. Therefore alternative methods to the diagonalization of this matrix should be used.

### 0.3 The Graph Convolution

Let   $f, g: V \rightarrow \mathbb{R}$, the convolution between the two functions is defined as:

$$
f \, * \, g = U^T\left(U f\odot U  g\right) \quad \left(= U_{ji}U_{jk}f_k U_{jl}g_l\right) \, ,
$$

where $\odot $ denotes the Hadamard product as usual. Note that this definition respects the Convolution Theorem: $\widehat{(f*g)} = \hat{f}\odot \hat{g}$. 

If we denote $\tilde{g} = U\cdot g$:
$$
\left(f \, * \, g\right)_i =  U_{ji}U_{jk}f_k \tilde{g}_j  =  U_{ji}\tilde{g}_j \delta_{jl}U_{lk}f_k  \iff f \, * \, g = 
U^T\cdot \begin{bmatrix}
    \tilde{g}_{0} & & 0 \\
    & \ddots & \\
    0 & & \tilde{g}_{n-1}
  \end{bmatrix}
  \cdot 
  U\cdot x\, .
$$

Because $\tilde{g}$ is the value of $g$ in the eigenbasis of the Laplacian, we can write $\tilde{g}_i = g(\lambda_i)$. In the literature they denote $\tilde{g}$ by $g_\theta$, but I donted it $\tilde{g}$ by convenience for the calculations with indices. This function $g_\theta$ is the filter. Finally we arrive at the final expression that is used in all the papers but it is generally not explained:

$$
x * g_\theta = U^T g_\theta(\Lambda) \,U x\, ,
$$

where again $\Lambda = \mathrm{diag}(\lambda_0, \dots, \lambda_{n-1})  $.

Therefore we end up with the general form for a convolutional layer in a graph:

$$
X^{(k+1)} = \phi \left( U^T \cdot g_{\theta}(\Lambda) \cdot U \cdot X^{(k)}\right) \,.
$$
Where in this case $X^{(k)} \in \mathbb{R}^{n\times d}$, so we assume that every node has $d$ features, and $X^k$ is just a matrix where each row is a $d$-dimensional feature-vector. 

Approximate the filter by Chebychsev polynomials:

$$
g_\theta(\Lambda) \approx \sum_{k=0}^K \theta_k T_k(\tilde{\Lambda})\, \implies x * g_\theta = U^T g_\theta(\Lambda) \,U x \approx \sum_{k=0}^K \theta_k \left(U^TT_k(\tilde{\Lambda})U \right)x =  \sum_{k=0}^K \theta_k T_k(\tilde{L}) x
$$

where $\tilde{\Lambda} = \frac{2}{\lambda_{max}}\Lambda - I_N$ and $\tilde{L} = \frac{2}{\lambda_{max}}L - I_N$.

Truncating the previous expansion at $K=1$ and *assuming* $\lambda_{max} \approx 2$, one finds:

$$
x * g_\theta \approx \theta_0 x + \theta_1 (L-I_N)x = \theta_0 x - \theta_1 D^{-1/2}AD^{-1/2} x
$$
Now in the paper of Kipf and Welling they make the arbitrary choice of $\theta \equiv \theta_0\stackrel{!}{=}-\theta_1$ further simplifying the convolution over the filter $g_\theta$ as:

$$
x * g_\theta \approx \theta \left(I_N + D^{-1/2}AD^{-1/2}\right) x
$$
Because the matrix inside the parenthesis has eigenvalues in the range $[0,2]$ they renormalize it once more:
$$
I_N + D^{-1/2}AD^{-1/2} \to \tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2} \, ,
$$
with
$$
\begin{align}
\tilde{D}_{ij}&\equiv \delta_{ij}\sum_k \tilde{A}_{ik}\, ,\\
\tilde{A} &\equiv A + I_N \, .
\end{align}
$$
We end up with:

$$
x * g_\theta \approx  \theta\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}x 
$$
which can be generalized, if instead of one dimensional features we have $d$ dimensional features, to:
$$
x * g_\theta  \approx \tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}X \Theta
$$

### 0.4 Bibliography:

- [Graph Convolutions and Machine Learning (Harvard)](https://dash.harvard.edu/bitstream/handle/1/38811540/OTNESS-SENIORTHESIS-2018.pdf?sequence=1&isAllowed=y)
- [Convolutional Neural Networks on Graphs with Fast Localized Spectral Filtering](https://papers.nips.cc/paper/6081-convolutional-neural-networks-on-graphs-with-fast-localized-spectral-filtering.pdf)
- [Graph convolutional networks: a comprehensive review](https://link.springer.com/article/10.1186/s40649-019-0069-y)
- [Spectral Networks and Deep Locally Connected Networks on Graphs](https://arxiv.org/pdf/1312.6203.pdf)
- [Introduction to Graph Neural Networks](https://www.morganclaypool.com/doi/abs/10.2200/S00980ED1V01Y202001AIM045)

## 1. GCN Layers in PyTorch

[PyTorch](https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html) defines **message passing** through the following equation:

$$
\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \right),
$$

with $\mathbf{x}^{(k-1)}_i \in \mathbb{R}^F$, $\mathbf{e}_{j,i} \in \mathbb{R}^D$ and:

- $\mathbf{x}^{(k-1)}_i \rightarrow  $ features of node i in layer (k-1).
- $\mathbf{e}_{j,i} \rightarrow $ edge features from node 𝑗 to node 𝑖.
- $\square \rightarrow$  denotes a differentiable, permutation invariant function, e.g., sum, mean or max.
- $\phi \rightarrow$ is a differentiable function such as MLPs (Multi Layer Perceptrons). In the [code implementation](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/conv/gcn_conv.html#GCNConv) of the GCN Layer it corresponds to the function **message()**.
- $\gamma \rightarrow$ is a differentiable function such as MLPs (Multi Layer Perceptrons). In the [code implementation](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/conv/message_passing.html#MessagePassing) of the GCN Layer it corresponds to the function **update()**.

For more clarity we could represent the indices not shown in the PyTorch docu as *greek letters*.

A simple example could be:
$
x^{(k)}_{i,\alpha} = \sigma\left(A_{ij}x^{(k-1)}_{j,\beta}W^{(k-1)}_{\beta\alpha}\right)
$

Where $\sigma$ is a non-linear activation function such as the ReLU function.

$$
x^{(k)}_{i,\alpha}
\quad
   \begin{cases}
      k & \text{is the k'th layer}\\
      i, & \text{is the number of nodes}\\
      \alpha, & \text{is the number of features}
    \end{cases}
$$

$A$ is a square matrix, because we want the preserve the number of nodes. $W$ instead can be a non square matrix, such that we change the number of features of the node from layer $k$ to layer $k+1$.


- $\square \rightarrow$  $Ax^{(k-1)}\left(= A_{ij}x^{(k-1)}_{j,\alpha}\right)$
- $\phi \rightarrow$ $\phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \equiv \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)}\right) = \sigma\left(Ax^{(k-1)}W^{(k-1)}\right)$
- $\gamma \rightarrow$ is just the identity function, basically it does nothing.

Where in the parenthesis I have included all the indices and used Einstein summation convention.

Lets work it out in the particular example of the GCN Layer:


$$
\mathbf{x}_i^{(k)} = \sum_{j \in \mathcal{N}(i) \cup \{ i \}} \frac{1}{\sqrt{\deg(i)} \cdot \sqrt{\deg(j)}} \cdot \left( \mathbf{\Theta} \cdot \mathbf{x}_j^{(k-1)} \right)\, ,\quad \left(=  \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}} x^{(k-1)}\mathbf{\Theta}\right)
$$

where $\deg(i)$ is the degree of the node $\rightarrow$ [Degree (graph theory)](https://en.wikipedia.org/wiki/Degree_(graph_theory))

- $\square \rightarrow$  $\sum_{j \in \mathcal{N}(i) \cup \{ i \}}$
- $\phi \rightarrow$ $\phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \equiv \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)}\right) =  \frac{1}{\sqrt{\deg(i)} \cdot \sqrt{\deg(j)}} \cdot \left( \mathbf{\Theta} \cdot \mathbf{x}_j^{(k-1)} \right)$
- $\gamma \rightarrow$ is just the identity function, basically it does nothing.

I am actually not sure about what $\phi$ does, because the normalization is done outside of *message* and the product with the weights $\mathbf{\Theta}$ as well (in the PyTorch code).

$$
\mathbf{x}_i^{(k)} = \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)}\right) = \sum_{j \in \mathcal{N}(i) \cup \{ i \}} \frac{1}{\sqrt{\deg(i)} \cdot \sqrt{\deg(j)}} \cdot \left( \mathbf{\Theta} \cdot \mathbf{x}_j^{(k-1)} \right),
$$

## 2. Exploring the dataset

In [5]:
le = preprocessing.LabelEncoder()
zkc = karate_club_graph()

data = from_networkx(zkc)
n_features = 6
embed = nn.Embedding(34, n_features)

data.x = embed.weight
data.y = torch.tensor(le.fit_transform(data.club), dtype=torch.float32)

In [8]:
# torch.unique(data.x)

In [9]:
data.x.histc(bins=2)

tensor([ 41., 163.], grad_fn=<HistcBackward>)

- Data structure:

In [10]:
data.__dict__

{'x': Parameter containing:
 tensor([[-0.7382, -0.2088,  0.5928,  0.8889,  0.5277,  0.5846],
         [ 0.4638,  2.4488,  0.1399, -0.3852, -1.8729,  1.4333],
         [ 1.1990, -0.6522,  0.1400,  0.6717, -0.1713, -1.9192],
         [-0.5701, -1.4612,  1.4591, -0.4475, -0.6011, -1.3340],
         [-1.5037,  0.8464,  0.8845,  1.6852, -0.6831, -1.5835],
         [-0.8618,  0.4507,  0.1794, -0.6434, -0.0695,  0.8467],
         [ 0.3148, -0.3778, -0.5804,  0.0696,  1.4144,  1.9650],
         [ 1.3317, -0.3174,  0.4669,  0.5679,  0.1118, -0.1033],
         [-0.2857, -0.0534, -0.4106, -0.4172,  0.2771, -0.4157],
         [ 1.5899, -0.1107, -0.3369,  0.8111,  0.1131, -0.7807],
         [-1.4079,  0.3540, -0.8098, -0.8489, -3.7270,  0.2566],
         [-0.5196, -0.2513, -0.2294, -0.7413, -0.1640, -0.2490],
         [-0.7793,  1.6842, -0.4517,  0.3681, -1.0456,  0.0626],
         [ 0.2841,  0.0798,  1.2007,  0.0566, -0.9146, -0.1025],
         [-1.6494,  0.4369,  0.3494,  1.0768,  0.4046,  0.4628

In [11]:
print('Node feature matrix with shape [num_nodes, num_node_features]: ', data.x.shape)
print('\nGraph connectivity in COO format with shape [2, num_edges]: ', data.edge_index.shape)
# print('\nNumber of classes: ', dataset.num_classes)

Node feature matrix with shape [num_nodes, num_node_features]:  torch.Size([34, 6])

Graph connectivity in COO format with shape [2, num_edges]:  torch.Size([2, 156])


In [6]:
max(data.edge_index[0]), max(data.edge_index[1])

(tensor(33), tensor(33))

##  3. The model in PyTorch

In [7]:
GCNConv??

In [8]:
class Net(torch.nn.Module):
    def __init__(self, n_features=2, n_hidden=2, n_output=1):
        super(Net, self).__init__()
        self.conv1 = GCNConv(n_features, n_hidden)
        self.conv2 = GCNConv(n_hidden, n_output)

    def forward(self, data):
        ''' x (Tensor): Node feature matrix. Shape [num_nodes, num_node_features]'''
        ''' edge_index (LongTensor): Graph connectivity in COO format. Shape [2, num_edges]'''
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.view(34, )

        return torch.sigmoid(x)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(n_features=n_features).to(device)
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), embed.parameters()), lr=0.01, weight_decay=5e-4)

In [135]:
model._modules['conv1']._parameters['weight'][:3]

tensor([[-0.0053, -0.0312,  0.0071, -0.0277,  0.0302, -0.0191,  0.0528,  0.0823,
         -0.0631,  0.0729,  0.0596,  0.0298,  0.0294, -0.0091,  0.0151,  0.0407],
        [-0.0509,  0.2361, -0.0555,  0.2472, -0.0025,  0.1550, -0.0704, -0.0828,
         -0.0008,  0.0061, -0.0575, -0.0340,  0.0319, -0.0304,  0.1290, -0.1260],
        [-0.0963,  0.0383, -0.0828, -0.0552,  0.0221,  0.0051,  0.0631,  0.1872,
         -0.0259, -0.0057,  0.1380,  0.1858, -0.1658,  0.0753, -0.0261, -0.0763]],
       grad_fn=<SliceBackward>)

In [22]:
model.train()
epochs = 800
writer = SummaryWriter("./runs/" + datetime.now().strftime("%Y%m%d-%H%M%S"))
save_statistics=False
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(data)
    # we are training with all the data at once, but for bigger datasets one can use a mask
    loss = F.binary_cross_entropy(out, data.y)
    acc = out.round().eq(data.y).sum().item()/len(data.y)
    if epoch % 10 == 0 and save_statistics:
        writer.add_scalar('training loss', loss, epoch)
        writer.add_scalar('Accuracy', acc, epoch)
    loss.backward()
    optimizer.step()

In [26]:
acc = model(data).round().eq(data.y).sum().item()/len(data.y)
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.9706


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=runs